In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

data = pd.read_json('characters_data.json')

data['name'] = data['name'].str.replace(r'\s*\(Don\'t Starve Together\)', '', regex=True)

ds_colors = {
    'background': '#2E2E2E',
    'text': '#E0E0E0',
    'plot_background': '#3E3E3E',
    'scatter': '#D2691E',
    'highlight': '#FF4500',
    'bar': '#8B4513',
    'grid': '#4A4A4A'
}

# Функция для отображения описания персонажа
def display_character_info(character_name):
    character = data[data['name'] == character_name].iloc[0]
    info = character['info']
    display(pd.Series(info).head(10))

# Функция для создания точечной диаграммы с подписями и подсветкой выбранного персонажа
def plot_sanity_vs_health(highlighted_character=None):
    plt.figure(figsize=(10, 6), facecolor=ds_colors['background'])
    health = data['stats'].apply(lambda x: x['Health'])
    sanity = data['stats'].apply(lambda x: x['Sanity'])
    names = data['name']

    scatter = plt.scatter(health, sanity, color=ds_colors['scatter'])

    for i, name in enumerate(names):
        plt.text(health[i], sanity[i], name, fontsize=9, ha='right', color=ds_colors['text'])

    # Подсветка выбранного персонажа
    if highlighted_character:
        character_index = data[data['name'] == highlighted_character].index[0]
        plt.scatter(health[character_index], sanity[character_index], color=ds_colors['highlight'], s=100)

    plt.title('Зависимость количества рассудка от здоровья', color=ds_colors['text'])
    plt.xlabel('Здоровье', color=ds_colors['text'])
    plt.ylabel('Рассудок', color=ds_colors['text'])
    plt.gca().set_facecolor(ds_colors['plot_background'])
    plt.gca().tick_params(colors=ds_colors['text'])
    plt.grid(color=ds_colors['grid'])
    plt.show()

# Функция для создания линейчатой диаграммы топ 4 персонажей по здоровью
def plot_top_4_health():
    data['Health'] = data['stats'].apply(lambda x: int(x['Health']))
    # Сортируем по здоровью и берем топ 4
    top_4 = data.sort_values(by='Health', ascending=False).head(4)
    plt.figure(figsize=(10, 6), facecolor=ds_colors['background'])
    plt.bar(top_4['name'], top_4['Health'], color='red')  # Изменение цвета столбцов на красный
    plt.title('Топ 4 персонажей по количеству здоровья', color=ds_colors['text'])
    plt.xlabel('Персонаж', color=ds_colors['text'])
    plt.ylabel('Здоровье', color=ds_colors['text'])
    plt.gca().set_facecolor(ds_colors['plot_background'])
    plt.gca().tick_params(colors=ds_colors['text'])
    plt.grid(color=ds_colors['grid'])
    plt.show()

# Функция для создания столбчатой диаграммы
def plot_stats(character_name):
    character = data[data['name'] == character_name].iloc[0]
    stats = character['stats']
    plt.figure(figsize=(10, 6), facecolor=ds_colors['background'])
    colors = ['red', 'orange', 'yellow']  # Цвета для столбцов
    plt.bar(stats.keys(), [int(v) for v in stats.values()], color=colors)  # Применение цветов
    plt.title(f'Статистика персонажа {character_name}', color=ds_colors['text'])
    plt.xlabel('Параметр', color=ds_colors['text'])
    plt.ylabel('Значение', color=ds_colors['text'])
    plt.gca().set_facecolor(ds_colors['plot_background'])
    plt.gca().tick_params(colors=ds_colors['text'])
    plt.grid(color=ds_colors['grid'])
    plt.show()

# Создание виджетов
character_dropdown = widgets.Dropdown(options=data['name'].tolist(), description='Персонаж:')
output = widgets.Output()

# Функция для обновления дашборда
def update_dashboard(change):
    with output:
        output.clear_output()
        display_character_info(change['new'])  # Используем change['new'] для получения нового значения
        plot_sanity_vs_health(highlighted_character=change['new'])
        plot_top_4_health()
        plot_stats(change['new'])

# Привязка функции к виджету
character_dropdown.observe(update_dashboard, names='value')

# Отображение дашборда
display(character_dropdown)
display(output)

# Инициализация первого отображения
update_dashboard({'new': character_dropdown.value})

Dropdown(description='Персонаж:', options=('Wilson', 'Willow', 'Wolfgang', 'Wendy', 'WX-78', 'Wickerbottom', '…

Output()